# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [ ]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

In [ ]:
import sys
import os
import sys
import json
import pickle
from sagemaker.pytorch import PyTorch
import sagemaker
import pandas as pd
from typing import Dict
from scripts.merge_leads_excerpts import get_training_dict

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [ ]:
sys.path.append('../../../../')
from deep.constants import *
from deep.utils import *

In [ ]:
%load_ext autoreload
%autoreload 2

# Data

Load data

In [ ]:
use_sample = True

DATA_PATH = os.path.join(
    "..", "..", "..", "..", "data", "frameworks_data", "data_v0.7.1"
)
EXCERPTS_PATH = os.path.join(DATA_PATH, "full_dataset_with_translations.csv")
LEADS_PATH = os.path.join(DATA_PATH, "leads_data.json")

data_folder = "data"
if use_sample:
    sample_percentage = 0.1
    data_file_name = "sample_data.csv"  # sample data
else:
    data_file_name = "full_data.csv"  # full data
    sample_percentage = None


# tbd each time to make sure changes in files are taken into account.
data_df = get_training_dict(
    leads_data_path=LEADS_PATH,
    excerpts_df_path=EXCERPTS_PATH,
    use_sample=use_sample,
    sample_percentage=sample_percentage,
)
data_df.to_csv(os.path.join(data_folder, data_file_name), index=None)

# do ths because
data_df = pd.read_csv(os.path.join(data_folder, data_file_name))


## Sagemaker Prep

### Session

Configure SageMaker

In [ ]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [ ]:
MLFLOW_SERVER

### send data to bucket

In [ ]:
job_name = f"pytorch-{formatted_time()}-entry-extraction"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_daR. Cherkita' / job_name  # Do not change this

data_path = str(input_path / 'data.pickle') # keep it as it is

# send data to s3 bucket
# need too check protocol, depending on data type (protocol 4 was made for pandas data inputs)

#data_df = get_df_from_dict(data)
data_df.to_pickle(data_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6

### Estimator Definition

In [ ]:
# GPU instances
instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

# CPU instances
instances = [
    'ml.c4.2xlarge',
    'ml.c4.4xlarge',
    'ml.c5n.2xlarge'
]

# https://aws.amazon.com/sagemaker/pricing/instance-types

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [ ]:
instance_type = "ml.p3.2xlarge"

experiment_name = "entry_extraction"
run_name = experiment_name  

hyperparameters = {
    "instance_type": instance_type,
    "tracking_uri": MLFLOW_SERVER,
    "experiment_name": experiment_name,
    "run_name": run_name,
    "model_name_or_path": "nreimers/mMiniLMv2-L6-H384-distilled-from-XLMR-Large",
    "tokenizer_name_or_path": "nreimers/mMiniLMv2-L6-H384-distilled-from-XLMR-Large",
    "learning_rate": 1e-4,
    "n_epochs": 1 if use_sample else 5,
    "weight_decay": 0.01,
    "dataloader_num_workers": 6,
    "val_batch_size": 8,
    "train_batch_size": 4,
    "max_len": 512,
    "extra_context_length": 64,
    "dropout": 0.2
    #"n_separate_layers": 1,
    #"per_device_train_batch_size": 1,
    #"per_device_eval_batch_size": 1,
    #"gradient_accumulation_steps": 8,
    #"save_strategy": "epoch",
    #"adam_beta1": 0.9,
    #"adam_beta2": 0.98,
    #"adam_epsilon": 1e-6,
    #"warmup_ratio": 0.3,
    #"fp16": true,
}

estimator = PyTorch(
    entry_point="train.py",
    source_dir=str(
        "scripts"
    ),
    output_path=str(DEV_BUCKET / "models/"),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py3",
    hyperparameters=hyperparameters,
    job_name=job_name,
    debugger_hook_config=False
    #     train_instance_count=2,
    #     train_instance_type="ml.c4.xlarge",
)

fit_arguments = {"train": str(input_path)}

In [ ]:
# Fit the estimator
estimator.fit(fit_arguments, job_name=job_name)